# PonyORM MDF Indexer

### Imports

In [1]:
import pony.orm
from pony.orm.core import EntityMeta
from pony import orm
import py
import time
import os

pony.orm.set_sql_debug(False)

# Database Setup

In [2]:
db = pony.orm.Database()

database_file = os.path.abspath('../mdf_index.sqlite')

db.bind(
    provider='sqlite',
    filename=database_file
)

class MDF(db.Entity):
    name = pony.orm.Required(
        str,
    )
    version = pony.orm.Required(
        str,
    )
    sha256 = pony.orm.Required(
        str, 
    )
    size = pony.orm.Optional(
        int, 
    )
    size_mb = pony.orm.Optional(
        float, 
    )
    atime = pony.orm.Optional(
        float,
    )
    channels = pony.orm.Set(
        'Channel',
    )
    
class Channel(db.Entity):
    name = pony.orm.Required(
        str,
        unique=True,
    )
    mdfs = pony.orm.Set(
        "MDF",
    )
    
db.generate_mapping(create_tables=True)

In [3]:
large_files = MDF.select().order_by(pony.orm.desc(MDF.size_mb))[0:5]
large_files

[MDF[3], MDF[5], MDF[8], MDF[10], MDF[11]]

In [4]:
for large_file in large_files:
    break

In [5]:
large_file.size_mb

0.9199695587158203

In [9]:
q = pony.orm.select((mdf.version, pony.orm.count(mdf)) for mdf in MDF)
q

In [10]:
q.show()

mdf.version|pony.orm.count(mdf)
-----------+-------------------
2.00       |102                
2.10       |97                 
2.14       |93                 
3.00       |98                 
3.10       |92                 
3.20       |102                
3.30       |113                
4.00       |106                
4.10       |93                 
4.11       |104                
